# Ensemble Learning and Random Forest 
Aggregating predictions of a group of predictors often generate better predictions. A group of predictors is called an ensemble.

### Voting Classifiers
A simple method to create a better classifier is to aggregate the predictions of each classifier by predicting the class that gets the most votes. This method is called a hard voting classifier.

In [43]:
# First let's load in the iris datsets
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

iris = load_iris()

# Combine into a single array 
data_all = np.column_stack((iris.data,iris.target)) # petal length and width and target

In [44]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(data_all, test_size = 0.2, random_state=42)
X_train = train_set[:, :4]
y_train = train_set[:, 4:]
X_test = test_set[:, :4]
y_test =test_set[:, 4:]

In [46]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()
svm_clf = SVC()

voting_clf = VotingClassifier(
    estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
    voting='hard'
)
voting_clf.fit(X_train, y_train.ravel()) # ravel() converts a column vector into a 1d array

VotingClassifier(estimators=[('lr', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)), ('rf', RandomF...,
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False))],
         n_jobs=1, voting='hard', weights=None)

In [47]:
# Let's look at each classifiers accuracy on the test set
from sklearn.metrics import accuracy_score
for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
    clf.fit(X_train, y_train.ravel())
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 1.0
RandomForestClassifier 1.0
SVC 1.0
VotingClassifier 1.0


In [28]:
# if all classifiers output class probabilities you can use soft voting, which predicts the class with the higest 
# class probability averaged over all the individual classifiers. This achieves a higher performance as it gives more weight to
# highly confident votes. To use this we change voting='soft'.

### Bagging and Pasting
Bagging is the method of training the same algorithm on different random subsets of the training set sampled with replacement.

Pasting is the method of training the same algorithm on different random subsets of the training set samepled without. replacement

In [48]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=500,
    max_samples=100, bootstrap=True,n_jobs=-1
)
bag_clf.fit(X_train, y_train.ravel())
y_pred = bag_clf.predict(X_test)

### Out-of-Bag Evaluation
With bagging some instances may be sampled several times, while some may never be sampled. By default only about 63% of training instances are sampled on for each predictor. The remaining 37% are called the out-of-bag (oob) instances.

In [49]:
# Here's an example - we just need to include oob_score = True
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=500,
    bootstrap=True, n_jobs=-1, oob_score = True
)
bag_clf.fit(X_train, y_train.ravel())
bag_clf.oob_score_

0.94999999999999996

In [50]:
# Now let's see what we get on the test set. We are expecting something around 94%
from sklearn.metrics import accuracy_score
y_pred = bag_clf.predict(X_test)
accuracy_score(y_test.ravel(), y_pred)

1.0

In [51]:
# The oob decision function for each training instance is also available. Since this a 3 class problem we recieve 3 probabilties
bag_clf.oob_decision_function_[:10]

array([[ 1.        ,  0.        ,  0.        ],
       [ 1.        ,  0.        ,  0.        ],
       [ 0.        ,  1.        ,  0.        ],
       [ 1.        ,  0.        ,  0.        ],
       [ 1.        ,  0.        ,  0.        ],
       [ 0.        ,  0.03333333,  0.96666667],
       [ 0.        ,  1.        ,  0.        ],
       [ 1.        ,  0.        ,  0.        ],
       [ 1.        ,  0.        ,  0.        ],
       [ 1.        ,  0.        ,  0.        ]])

### Random Forests
Random Forests are a decision tree ensemble generally trained via the bagging method. Rather than building a bagging classifer and passing it a decision tree classifer, Random Forest is optimized and more convenient.

The Random Forest introduces extra randomness when growing trees; instead of searching for best feature when spliting, it searchs for the best feature amoung a random subset of features. This creates extra diverity, which again trades higher bias for lower variance yielding overall better performance.

In [52]:
from sklearn.ensemble import RandomForestClassifier

rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=-1)
rnd_clf.fit(X_train, y_train.ravel())

y_pred = rnd_clf.predict(X_test)

### Extra-Trees
When growing a tree in a Random Forest it's possible to make the trees more random by also selecting a random threshold for each feature rather than searching for the best possible thresholds (like decision trees). These are called Extremely Randomized Trees and again trade more bias for a lower variance. These are faster to train as finding the optimal threshold for each feature at each node is the  most time consuming task.

Sklearns ExtraTreesClassifier can be used instead of the RandomForestClassifer class.

### Feature Importance
Random Forests make it easy to measure the relative importance of each feature. This calculated (in sklearn) by looking at how much the tree nodes that use that feature reduce the impurity on average (across all the trees in the forest). It's a weighted average where each node's weight is equal to the number of training samples associated with it. 

The scores are then scaled to sum up to 1.

In [53]:
for name, score in zip(iris["feature_names"], rnd_clf.feature_importances_):
    print(name, score)

sepal length (cm) 0.101003229869
sepal width (cm) 0.0292191422065
petal length (cm) 0.4621493654
petal width (cm) 0.407628262525


### Boosting 
Boosing refers to any ensemble method that can combine several week learners into a strong learner. The idea od most boostind methods is to train predictors sequentially, with each trying to correct its predecessors mistake. The most popular boosting methods are AdaBoost and Gradient Boosting.

### AdaBoost
In AdaBoosting first a base classifier is trained and used to make predictions on the training set. The relative weight of misclassified training instances is then increased. A second classifier is trained using the updated weights and again makes predictions on the training set, weights are then updated, and so on.

Once all predictors are trained it makes predictions like bagging or pasting, except that predictors have different weights depending on their overall accuracy on the weighted training set.

In AdaBoost first each instance of the training set is assigned a weight equal to 1/m, where m is number of instances. A frist predictor is then trained and its weighted error rate r is computed. The predictors weight alpha is then computed. An accurate predictor recieves a higher weight. A random predictor will get a weight close to zero, and a worse than random will get a negative weight.

Next the instance weights are updated by boosting the weight of the misclassified instances. The weights are then normalized to sum up to 1.

Next a new predictor is trained using the updated weights, and the whole process continues untill a set number of predictors or a prefect predictor is found.

To make predictions AdaBoost computes the predictions for all predictors and weighs them using the predictor weights alpha. The predicted class determined by majority vote.

In [56]:
# Here's an example with sklearn. 
# Sklearn actually uses a multiclass version of AdaBoost called SAMME (Stagewise Additive Modelling using a Multiclass 
# Expoenential loss function). If the predictors can estimate class probabilties sklearn can use a variant of SAMME 
# called SAMME.R, where R stands for Real. This generally performs better.

from sklearn.ensemble import AdaBoostClassifier

ada_clf = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1), 
    n_estimators=200,algorithm = "SAMME.R", learning_rate=0.5
)
ada_clf.fit(X_train, y_train.ravel())

AdaBoostClassifier(algorithm='SAMME.R',
          base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=1,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best'),
          learning_rate=0.5, n_estimators=200, random_state=None)

### Gradient Boosting
Gradient Boosting also works sequentially adding predictors to an ensemble to correct its predecessor, however rather than updating instance weights it fits new predictors to the residual errors of the previous predictor.


In [59]:
## Here's a regression example using a decision tree base predictor.

from sklearn.tree import DecisionTreeRegressor

tree_reg1 = DecisionTreeRegressor(max_depth=2)
tree_reg1.fit(X_train, y_train.ravel())

DecisionTreeRegressor(criterion='mse', max_depth=2, max_features=None,
           max_leaf_nodes=None, min_impurity_split=1e-07,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, presort=False, random_state=None,
           splitter='best')

In [69]:
y_train[:5].ravel()

array([ 0.,  0.,  1.,  0.,  0.])

In [73]:
# Now train a second tree on the residual errors made by the first predictor.
y2 = y_train.ravel() - tree_reg1.predict(X_train)
tree_reg2 = DecisionTreeRegressor(max_depth=2)
tree_reg2.fit(X_train, y2)
#y2[:5]

DecisionTreeRegressor(criterion='mse', max_depth=2, max_features=None,
           max_leaf_nodes=None, min_impurity_split=1e-07,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, presort=False, random_state=None,
           splitter='best')

In [74]:
# Now train a third regressor on the residual errors made by the second predictor.
y3 = y2 - tree_reg2.predict(X_train)
tree_reg3 = DecisionTreeRegressor(max_depth = 2)
tree_reg3.fit(X_train, y3)

DecisionTreeRegressor(criterion='mse', max_depth=2, max_features=None,
           max_leaf_nodes=None, min_impurity_split=1e-07,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, presort=False, random_state=None,
           splitter='best')

In [75]:
# Now we have an ensemble containing three trees. To make predictors simply add up the predictors of all the trees.
y_pred = sum(tree.predict(X_test) for tree in (tree_reg1, tree_reg2, tree_reg3))

In [76]:
# Sklearn contains a GradientBoostingRegressor class that makes this process simplier.
# Here's the equilavent code
from sklearn.ensemble import GradientBoostingRegressor

gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=3, learning_rate=1.0)
gbrt.fit(X_train, y_train.ravel())

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=1.0, loss='ls', max_depth=2, max_features=None,
             max_leaf_nodes=None, min_impurity_split=1e-07,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=3, presort='auto',
             random_state=None, subsample=1.0, verbose=0, warm_start=False)

The learning rate hyperparameter scales the contribution of each tree. A low value will need more trees but will usually generalize better. This regularization technique is called shrikage.

To find the optimal number of trees 'early stopping' can be used. This can be done by using staged_predict(), it will iterate over the predictions made by the ensemble at each stage of training (1 tree, 2 trees, 3 trees, etc.). The validation error is commputed at each training stage to find the optimal number of trees. Once the optimal is found another GBRT ensemble is trained usinf the optimal parameters. 

In [82]:
from sklearn.metrics import mean_squared_error

X_val = X_test
y_val = y_test # for naming demonstration perposes only.

gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=120)
gbrt.fit(X_train, y_train.ravel())

errors = [mean_squared_error(y_val, y_pred)
         for y_pred in gbrt.staged_predict(X_val)]
bst_n_estimators = np.argmin(errors)

gbrt_best = GradientBoostingRegressor(max_depth=2, n_estimators=bst_n_estimators)
gbrt_best.fit(X_train, y_train.ravel())

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=2, max_features=None,
             max_leaf_nodes=None, min_impurity_split=1e-07,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=34, presort='auto',
             random_state=None, subsample=1.0, verbose=0, warm_start=False)

In [83]:
# It is also possible to implement early stopping by actually stopping training earlt instead of training a large number of trees
# and looking back to find the optimal number.

# To do this we set warm_start=True, which makes sklearn keep existing trees when the fit() method is called, allowing 
# incremental training.

gbrt = GradientBoostingRegressor(max_depth = 2, warm_start=True)

min_val_error = float("inf")
error_going_up = 0
for n_estimators in range(1, 120):
    gbrt.n_estimators = n_estimators
    gbrt.fit(X_train, y_train.ravel())
    y_pred = gbrt.predict(X_val)
    val_error = mean_squared_error(y_val, y_pred)
    if val_error < min_val_error:   # keep the best here
        min_val_error = val_error
        error_going_up = 0
    else:
        error_going_up += 1
        if error_going_up == 5:
            break  # early stop

### Stacking
Stacking (short for stacked generalization) is when a model is used perform model aggregation (rather than a simple hard voting or average function). To perform model stacking the training set is split in subsets. The first subset is used to train the level one predictors. Next, the first layer predictors are used to make predictions on the secod set. This ensures predictions are 'clean', since predictors never saw these instances during training. Finally a blender or meta layer predictor is trained on the new trainined set so it learns to predict the target value given the first layer.

Multiple (k) meta layer predictors can be used, the trick is to split the training set into k subsets. 